Before Start working, we should has a overview for the data  

| variables | definition | key |  
|-----------|:----------:|:---:|
| survival | Survival	| 0 = No, 1 = Yes |  
| pclass  | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |  
| sex	| Sex |  |  
| Age	| Age in years |  |  
| sibsp | # of siblings / spouses aboard the Titanic	|  |  
| parch | # of parents / children aboard the Titanic	|  |  
| ticket | Ticket number | |  
| fare | Passenger fare | |  
| cabin | Cabin number | |  
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |  

Now, let import the data and get some sample of it  

In [1]:

import pandas as pd
data_train = pd.read_csv('../input/train.csv')
data_test = pd.read_csv('../input/test.csv')
data_train.sample(5)
data_columns = data_train.columns
for column in data_columns:
    print('is {column} has invalid data? {isna}'.format(column=column, isna=pd.isna(data_train[[column]]).any()))

import tensorflow as tf

is PassengerId has invalid data? PassengerId    False
dtype: bool
is Survived has invalid data? Survived    False
dtype: bool
is Pclass has invalid data? Pclass    False
dtype: bool
is Name has invalid data? Name    False
dtype: bool
is Sex has invalid data? Sex    False
dtype: bool
is Age has invalid data? Age    True
dtype: bool
is SibSp has invalid data? SibSp    False
dtype: bool
is Parch has invalid data? Parch    False
dtype: bool
is Ticket has invalid data? Ticket    False
dtype: bool
is Fare has invalid data? Fare    False
dtype: bool
is Cabin has invalid data? Cabin    True
dtype: bool
is Embarked has invalid data? Embarked    True
dtype: bool


ImportError: Could not find 'cudnn64_7.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Note that installing cuDNN is a separate step from installing CUDA, and this DLL is often found in a different directory from the CUDA DLLs. You may install the necessary DLL by downloading cuDNN 7 from this URL: https://developer.nvidia.com/cudnn